# Lista todos os jogos Regulares da Temporada 22/23

In [579]:
#07/05/2023

import pandas as pd
import numpy as np 
from nba_api.stats.endpoints import leaguegamefinder

pd.set_option('display.max_columns', 500)

season = '2022-23'
season_type = 'Regular Season'
league_id = '00' #00 = NBA, pois a consulta retorna dados de outras ligas tb


gamefinder = leaguegamefinder.LeagueGameFinder (
    season_nullable=season,
    season_type_nullable = season_type,
    league_id_nullable = league_id
)
          
# The first DataFrame of those returned is what we want.
df_TodosJogos = gamefinder.get_data_frames()[0]

season = '2021-22'
season_type = 'Regular Season'
league_id = '00' #00 = NBA, pois a consulta retorna dados de outras ligas tb


gamefinder = leaguegamefinder.LeagueGameFinder (
    season_nullable=season,
    season_type_nullable = season_type,
    league_id_nullable = league_id
)
          
# The first DataFrame of those returned is what we want.
df_TodosJogos2 = gamefinder.get_data_frames()[0]

df_TodosJogos = pd.concat([df_TodosJogos, df_TodosJogos2])

df_TodosJogos = df_TodosJogos.sort_values(by=['GAME_DATE','GAME_ID'], ascending=False, inplace=False)
df_TodosJogos.head(10)


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
16,22022,1610612757,POR,Portland Trail Blazers,0022201230,2023-04-09,POR vs. GSW,L,241,101,38,95,0.400,9,35,0.257,16,24,0.667,6,24,30,25,5,3,14,9,-56.0
19,22022,1610612744,GSW,Golden State Warriors,0022201230,2023-04-09,GSW @ POR,W,240,157,58,96,0.604,27,49,0.551,14,16,0.875,9,49,58,47,13,6,15,18,56.0
2,22022,1610612746,LAC,LA Clippers,0022201229,2023-04-09,LAC @ PHX,W,240,119,49,100,0.490,5,23,0.217,16,24,0.667,14,39,53,22,3,5,10,18,5.0
6,22022,1610612756,PHX,Phoenix Suns,0022201229,2023-04-09,PHX vs. LAC,L,240,114,42,99,0.424,14,37,0.378,16,22,0.727,12,35,47,29,4,3,7,21,-5.0
23,22022,1610612762,UTA,Utah Jazz,0022201228,2023-04-09,UTA @ LAL,L,240,117,51,101,0.505,10,35,0.286,5,6,0.833,7,38,45,31,4,4,11,16,-11.0
29,22022,1610612747,LAL,Los Angeles Lakers,0022201228,2023-04-09,LAL vs. UTA,W,240,128,47,97,0.485,18,45,0.400,16,21,0.762,10,39,49,28,4,7,10,9,11.0
9,22022,1610612758,SAC,Sacramento Kings,0022201227,2023-04-09,SAC @ DEN,L,241,95,37,85,0.435,11,43,0.256,10,15,0.667,10,24,34,24,7,6,15,18,-14.0
25,22022,1610612743,DEN,Denver Nuggets,0022201227,2023-04-09,DEN vs. SAC,W,240,109,42,84,0.500,7,28,0.250,18,25,0.720,15,36,51,25,11,2,16,15,14.0
4,22022,1610612763,MEM,Memphis Grizzlies,0022201226,2023-04-09,MEM @ OKC,L,241,100,40,92,0.435,7,37,0.189,13,18,0.722,11,32,43,25,8,4,12,16,-15.0
13,22022,1610612760,OKC,Oklahoma City Thunder,0022201226,2023-04-09,OKC vs. MEM,W,240,115,45,90,0.500,12,36,0.333,13,16,0.813,8,41,49,30,5,2,14,17,15.0


In [580]:
df_TodosJogos.shape

(4920, 28)

In [581]:
# Função que Cria um registro único, adicionando o sufixo _HOME nas estatísticas do time da casa e _AWAY no visitante

def combine_team_games(df):
    # Join every row to all others with the same game ID.
    joined = pd.merge(df, df, suffixes=['_HOME', '_AWAY'],
                          on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
 
    # Filter out any row that is joined to itself.
    result = joined[joined.TEAM_ID_HOME != joined.TEAM_ID_AWAY]

    # Keep rows where TEAM_A is the home team
    #Filtra somente o registro que contém o "vs." que indica que é o time que jogou em casa
    result = result[result.MATCHUP_HOME.str.contains(' vs. ')]
    
    return result

# Criando a ABT

In [582]:
cont = 0

df_abt = pd.DataFrame()
game_id_processado = 0

for row in df_TodosJogos.itertuples():
    game_id = row.GAME_ID

    #Verifica se já processou esse ID_Game
    if game_id_processado == game_id:
            continue
    else:
        game_id_processado = game_id
    
    # Filtra apenas 1 confronto
    df_Confronto = pd.DataFrame()
    df_Confronto = df_TodosJogos[df_TodosJogos.GAME_ID == game_id]
   
    #df_ConfrontoUnificado = pd.DataFrame()
    df_ConfrontoUnificado = combine_team_games(df_Confronto)
    
    #print(df_ConfrontoUnificado)
    df_abt = pd.concat([df_abt, df_ConfrontoUnificado],ignore_index=True)
    
    #cont = cont + 1
    #if cont == 3:
    #    break

In [583]:
df_abt

,SEASON_ID,TEAM_ID_HOME,TEAM_ABBREVIATION_HOME,TEAM_NAME_HOME,GAME_ID,GAME_DATE,MATCHUP_HOME,WL_HOME,MIN_HOME,PTS_HOME,FGM_HOME,FGA_HOME,FG_PCT_HOME,FG3M_HOME,FG3A_HOME,FG3_PCT_HOME,FTM_HOME,FTA_HOME,FT_PCT_HOME,OREB_HOME,DREB_HOME,REB_HOME,AST_HOME,STL_HOME,BLK_HOME,TOV_HOME,PF_HOME,PLUS_MINUS_HOME,TEAM_ID_AWAY,TEAM_ABBREVIATION_AWAY,TEAM_NAME_AWAY,MATCHUP_AWAY,WL_AWAY,MIN_AWAY,PTS_AWAY,FGM_AWAY,FGA_AWAY,FG_PCT_AWAY,FG3M_AWAY,FG3A_AWAY,FG3_PCT_AWAY,FTM_AWAY,FTA_AWAY,FT_PCT_AWAY,OREB_AWAY,DREB_AWAY,REB_AWAY,AST_AWAY,STL_AWAY,BLK_AWAY,TOV_AWAY,PF_AWAY,PLUS_MINUS_AWAY
0,22022,1610612757,POR,Portland Trail Blazers,0022201230,2023-04-09,POR vs. GSW,L,241,101,38,95,0.400,9,35,0.257,16,24,0.667,6,24,30,25,5,3,14,9,-56.0,1610612744,GSW,Golden State Warriors,GSW @ POR,W,240,157,58,96,0.604,27,49,0.551,14,16,0.875,9,49,58,47,13,6,15,18,56.0
1,22022,1610612756,PHX,Phoenix Suns,0022201229,2023-04-09,PHX vs. LAC,L,240,114,42,99,0.424,14,37,0.378,16,22,0.727,12,35,47,29,4,3,7,21,-5.0,1610612746,LAC,LA Clippers,LAC @ PHX,W,240,119,49,100,0.490,5,23,0.217,16,24,0.667,14,39,53,22,3,5,10,18,5.0
2,22022,1610612747,LAL,Los Angeles Lakers,0022201228,2023-04-09,LAL vs. UTA,W,240,128,47,97,0.485,18,45,0.400,16,21,0.762,10,39,49,28,4,7,10,9,11.0,1610612762,UTA,Utah Jazz,UTA @ LAL,L,240,117,51,101,0.505,10,35,0.286,5,6,0.833,7,38,45,31,4,4,11,16,-11.0
3,22022,1610612743,DEN,Denver Nuggets,0022201227,2023-04-09,DEN vs. SAC,W,240,109,42,84,0.500,7,28,0.250,18,25,0.720,15,36,51,25,11,2,16,15,14.0,1610612758,SAC,Sacramento Kings,SAC @ DEN,L,241,95,37,85,0.435,11,43,0.256,10,15,0.667,10,24,34,24,7,6,15,18,-14.0
4,22022,1610612760,OKC,Oklahoma City Thunder,0022201226,2023-04-09,OKC vs. MEM,W,240,115,45,90,0.500,12,36,0.333,13,16,0.813,8,41,49,30,5,2,14,17,15.0,1610612763,MEM,Memphis Grizzlies,MEM @ OKC,L,241,100,40,92,0.435,7,37,0.189,13,18,0.722,11,32,43,25,8,4,12,16,-15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,22021,1610612752,NYK,New York Knicks,0022100005,2021-10-20,NYK vs. BOS,W,288,138,51,105,0.486,17,45,0.378,19,27,0.704,7,48,55,27,9,10,17,22,4.0,1610612738,BOS,Boston Celtics,BOS @ NYK,L,290,134,48,117,0.410,21,57,0.368,17,23,0.739,15,41,56,34,13,9,18,24,-4.0
2456,22021,1610612765,DET,Detroit Pistons,0022100004,2021-10-20,DET vs. CHI,L,241,88,36,90,0.400,6,28,0.214,10,13,0.769,11,36,47,17,7,5,16,16,-6.0,1610612741,CHI,Chicago Bulls,CHI @ DET,W,239,94,37,86,0.430,7,23,0.304,13,15,0.867,9,39,48,18,8,5,17,19,6.0
2457,22021,1610612766,CHA,Charlotte Hornets,0022100003,2021-10-20,CHA vs. IND,W,239,123,46,107,0.430,13,31,0.419,18,27,0.667,12,34,46,29,9,5,8,21,1.0,1610612754,IND,Indiana Pacers,IND @ CHA,L,240,122,42,90,0.467,17,47,0.362,21,24,0.875,8,43,51,29,2,10,16,24,-1.0
2458,22021,1610612747,LAL,Los Angeles Lakers,0022100002,2021-10-19,LAL vs. GSW,L,241,114,45,95,0.474,15,42,0.357,9,19,0.474,5,40,45,21,7,4,17,25,-7.0,1610612744,GSW,Golden State Warriors,GSW @ LAL,W,240,121,41,93,0.441,14,39,0.359,25,30,0.833,9,41,50,30,9,2,17,18,7.0


In [584]:
confronto = 'POR vs. GSW'
df_abt.query('MATCHUP_HOME == @confronto').sort_values(by=['GAME_DATE'], ascending=False).head(5)
    


,SEASON_ID,TEAM_ID_HOME,TEAM_ABBREVIATION_HOME,TEAM_NAME_HOME,GAME_ID,GAME_DATE,MATCHUP_HOME,WL_HOME,MIN_HOME,PTS_HOME,FGM_HOME,FGA_HOME,FG_PCT_HOME,FG3M_HOME,FG3A_HOME,FG3_PCT_HOME,FTM_HOME,FTA_HOME,FT_PCT_HOME,OREB_HOME,DREB_HOME,REB_HOME,AST_HOME,STL_HOME,BLK_HOME,TOV_HOME,PF_HOME,PLUS_MINUS_HOME,TEAM_ID_AWAY,TEAM_ABBREVIATION_AWAY,TEAM_NAME_AWAY,MATCHUP_AWAY,WL_AWAY,MIN_AWAY,PTS_AWAY,FGM_AWAY,FGA_AWAY,FG_PCT_AWAY,FG3M_AWAY,FG3A_AWAY,FG3_PCT_AWAY,FTM_AWAY,FTA_AWAY,FT_PCT_AWAY,OREB_AWAY,DREB_AWAY,REB_AWAY,AST_AWAY,STL_AWAY,BLK_AWAY,TOV_AWAY,PF_AWAY,PLUS_MINUS_AWAY
0,22022,1610612757,POR,Portland Trail Blazers,0022201230,2023-04-09,POR vs. GSW,L,241,101,38,95,0.400,9,35,0.257,16,24,0.667,6,24,30,25,5,3,14,9,-56.0,1610612744,GSW,Golden State Warriors,GSW @ POR,W,240,157,58,96,0.604,27,49,0.551,14,16,0.875,9,49,58,47,13,6,15,18,56.0
401,22022,1610612757,POR,Portland Trail Blazers,0022200830,2023-02-08,POR vs. GSW,W,241,125,43,78,0.551,8,27,0.296,31,32,0.969,12,35,47,25,6,3,21,16,3.0,1610612744,GSW,Golden State Warriors,GSW @ POR,L,240,122,46,97,0.474,21,49,0.429,9,10,0.900,10,22,32,31,11,2,11,24,-3.0
1571,22021,1610612757,POR,Portland Trail Blazers,0022100894,2022-02-24,POR vs. GSW,L,241,95,31,79,0.392,9,35,0.257,24,29,0.828,6,25,31,16,5,4,18,29,-37.0,1610612744,GSW,Golden State Warriors,GSW @ POR,W,240,132,45,88,0.511,15,41,0.366,27,32,0.844,10,36,46,36,10,7,12,23,37.0


In [585]:
df_abt = df_abt.drop('MIN_HOME', axis='columns')
df_abt = df_abt.drop('MIN_AWAY', axis='columns')
df_abt = df_abt.drop('FGA_HOME', axis='columns') #Total de tentativas de cestas de 2 pts
df_abt = df_abt.drop('FGA_AWAY', axis='columns') #Total de tentativas de cestas de 2 pts
df_abt = df_abt.drop('FG3A_HOME', axis='columns') #Total de tentativas de cestas de 3 pts
df_abt = df_abt.drop('FG3A_AWAY', axis='columns') #Total de tentativas de cestas de 3 pts
df_abt = df_abt.drop('FTA_HOME', axis='columns') #Total de tentativas de cestas de 1 pts
df_abt = df_abt.drop('FTA_AWAY', axis='columns') #Total de tentativas de cestas de 1 pts
df_abt = df_abt.drop('REB_HOME', axis='columns') #Total de Rebotes
df_abt = df_abt.drop('REB_AWAY', axis='columns') #Total de Rebotes
df_abt = df_abt.drop('AST_HOME', axis='columns') #Total de Assistências
df_abt = df_abt.drop('AST_AWAY', axis='columns') #Total de Assistências
df_abt = df_abt.drop('STL_HOME', axis='columns') #Roubadas de bola 
df_abt = df_abt.drop('STL_AWAY', axis='columns') #Roubadas de bola
df_abt = df_abt.drop('BLK_HOME', axis='columns') #Tocos
df_abt = df_abt.drop('BLK_AWAY', axis='columns') #Tocos
df_abt = df_abt.drop('TOV_HOME', axis='columns') #Qtd Vezes que perdeu a bola para o adversário
df_abt = df_abt.drop('TOV_AWAY', axis='columns') #Qtd Vezes que perdeu a bola para o adversário
df_abt = df_abt.drop('PF_HOME', axis='columns') #Qtd de Faltas
df_abt = df_abt.drop('PF_AWAY', axis='columns') #Qtd de Faltas
df_abt = df_abt.drop('WL_AWAY', axis='columns')  # W/L do time de fora


In [586]:
df_abt.shape

(2460, 32)

In [587]:
#Início e Fim da Temporada regular
df_abt.agg({'GAME_DATE': ['min', 'max']}) 

,GAME_DATE
min,2021-10-19
max,2023-04-09


In [588]:
#Criar duas variáveis na ABT que irá analisar os últimos 5 confrontos entre os times e irá dizer W/L e Tot. Vitória e Tot. Derrota
cont = 0
for row in df_abt.itertuples():
    Team_id_home = row.TEAM_ID_HOME
    Team_id_away = row.TEAM_ID_AWAY
    confronto = row.MATCHUP_HOME
    
    df_confronto = df_abt.query('MATCHUP_HOME == @confronto and GAME_DATE < @row.GAME_DATE').sort_values(by=['GAME_DATE'], ascending=False).head(5)
    
    #if len(df_confronto) > 1:
     #  print(len(df_confronto))
    
    if len(df_confronto) > 0:
        qtd_Derrota = 0
        qtd_Vitoria = 0

        for row_2 in df_confronto.itertuples():
            if row_2.WL_HOME == 'W':
                qtd_Vitoria = qtd_Vitoria + 1
                print (qtd_Vitoria)
            else:
                qtd_Derrota = qtd_Derrota + 1
                print (qtd_Derrota)
       
        df_abt.loc[row.Index, ['WL_TOTVIT_CONFRONTO_HOME']] = int(qtd_Vitoria)
        df_abt.loc[row.Index, ['WL_TOTVIT_CONFRONTO_AWAY']] = int(qtd_Derrota)
    else:
        if row.WL_HOME == 'W':
            df_abt.loc[row.Index, ['WL_TOTVIT_CONFRONTO_HOME']] = 1  
            df_abt.loc[row.Index, ['WL_TOTVIT_CONFRONTO_AWAY']] = 0 
        else:
            df_abt.loc[row.Index, ['WL_TOTVIT_CONFRONTO_HOME']] = 0
            df_abt.loc[row.Index, ['WL_TOTVIT_CONFRONTO_AWAY']] = 1 

    #break

    #Soma o total de Votórias e Total de Derrotas na Temporada, tanto do time Home, quanto do time Away
    #Caso o time não tiver 10 jogos na temporada, será pesquisado na temporada passada
    
    #Processamento HOME TEAM
    Team_id_home = row.TEAM_ID_HOME
    df_ultimosjogos_Home = df_abt.query('TEAM_ID_HOME == @Team_id_home and GAME_DATE < @row.GAME_DATE').sort_values(by=['GAME_DATE'], ascending=False)
    
    if len(df_ultimosjogos_Home) > 0:
        df_temp = df_ultimosjogos_Home.groupby(['WL_HOME'])['SEASON_ID'].count().rename('TOTAL').reset_index()
        
        if df_temp.loc[df_temp['WL_HOME'] == 'W'].size > 0:
            TotalVitoriasHome = df_temp.loc[df_temp['WL_HOME'] == 'W']['TOTAL'].values[0]
        else:
            TotalVitoriasHome = 0
        
        if df_temp.loc[df_temp['WL_HOME'] == 'L'].size > 0:
            TotalDerrotasHome = df_temp.loc[df_temp['WL_HOME'] == 'L']['TOTAL'].values[0]
        else:
            TotalDerrotasHome = 0    

        df_abt.loc[row.Index, ['WL_TOTVIT_TEMPORADA_HOME']] = int(TotalVitoriasHome) 
        df_abt.loc[row.Index, ['WL_TOTDER_TEMPORADA_HOME']] = int(TotalDerrotasHome) 
    else:
        if row.WL_HOME == 'W':
            df_abt.loc[row.Index, ['WL_TOTVIT_TEMPORADA_HOME']] = 1
            df_abt.loc[row.Index, ['WL_TOTDER_TEMPORADA_HOME']] = 0
        else: 
            df_abt.loc[row.Index, ['WL_TOTVIT_TEMPORADA_HOME']] = 0
            df_abt.loc[row.Index, ['WL_TOTDER_TEMPORADA_HOME']] = 1
        
    #Processamento AWAY TEAM
    Team_id_away = row.TEAM_ID_AWAY
    df_ultimosjogos_Away = df_abt.query('TEAM_ID_AWAY == @Team_id_away and GAME_DATE < @row.GAME_DATE').sort_values(by=['GAME_DATE'], ascending=False)
    
    if len(df_ultimosjogos_Away) > 0:
        df_temp = df_ultimosjogos_Away.groupby(['WL_HOME'])['SEASON_ID'].count().rename('TOTAL').reset_index()
        
        if df_temp.loc[df_temp['WL_HOME'] == 'W'].size > 0:
            TotalVitoriasAway = df_temp.loc[df_temp['WL_HOME'] == 'W']['TOTAL'].values[0]
        else:    
            TotalVitoriasAway = 0
            
        if  df_temp.loc[df_temp['WL_HOME'] == 'L'].size > 0:   
            TotalDerrotasAway = df_temp.loc[df_temp['WL_HOME'] == 'L']['TOTAL'].values[0]
        else:
            TotalDerrotasAway = 0
        
        df_abt.loc[row.Index, ['WL_TOTVIT_TEMPORADA_AWAY']] = int(TotalVitoriasAway) 
        df_abt.loc[row.Index, ['WL_TOTDER_TEMPORADA_AWAY']] = int(TotalDerrotasAway) 
    else:
        if row.WL_HOME == 'W':
            df_abt.loc[row.Index, ['WL_TOTVIT_TEMPORADA_AWAY']] = 0
            df_abt.loc[row.Index, ['WL_TOTDER_TEMPORADA_AWAY']] = 1
        else:
            df_abt.loc[row.Index, ['WL_TOTVIT_TEMPORADA_AWAY']] = 1
            df_abt.loc[row.Index, ['WL_TOTDER_TEMPORADA_AWAY']] = 0
    #print(TotalVitoriasHome)
    #print(TotalDerrotasHome)
    #break

1
1
x
1
1
2
x
1
1
2
x
1
2
x
1
1
x
1
1
x
1
2
3
x
1
2
3
x
1
x
1
1
x
1
2
3
x
1
2
3
x
1
1
2
x
1
2
1
x
1
2
x
1
2
x
1
1
x
1
2
3
x
1
1
2
x
1
1
2
x
1
x
1
x
1
x
1
1
2
x
1
2
1
x
1
1
x
1
1
2
x
1
1
2
x
1
x
1
2
1
x
1
2
3
x
1
1
2
x
1
1
2
x
1
1
x
1
2
3
x
1
2
x
1
1
2
x
1
1
2
x
1
1
x
1
1
2
x
1
1
2
x
1
2
3
x
1
2
x
1
2
x
1
2
x
1
2
1
x
1
2
3
x
1
2
x
1
2
3
x
1
2
1
x
1
x
1
1
x
1
x
1
1
2
x
1
1
2
x
1
1
2
x
1
2
x
1
2
3
x
1
2
1
x
1
2
1
x
1
2
x
1
1
2
x
1
1
2
x
1
x
1
2
3
x
1
x
1
x
1
1
x
1
2
x
1
x
1
1
x
1
1
2
x
1
1
2
x
1
2
3
x
1
1
2
x
1
x
1
2
x
1
2
x
1
x
1
2
3
x
1
1
2
x
1
x
1
1
x
1
2
1
x
1
2
3
x
1
1
x
1
2
x
1
2
1
x
1
x
1
2
x
1
x
1
x
1
1
x
1
x
1
2
3
x
1
2
x
1
x
1
x
1
1
2
x
1
2
3
x
1
1
x
1
x
1
1
2
x
1
2
x
1
x
1
1
2
x
1
x
1
x
1
2
3
x
1
2
3
x
1
2
3
x
1
1
x
1
2
x
1
x
1
x
1
x
1
x
1
x
1
1
2
x
1
1
2
x
1
x
1
x
1
2
1
x
1
2
x
1
1
2
x
1
2
3
x
1
x
1
x
1
x
1
x
1
2
3
x
1
1
2
x
1
x
1
1
2
x
1
1
2
x
1
x
1
1
2
x
1
2
3
x
1
2
x
1
1
2
x
1
x
1
x
1
2
x
1
2
3
x
1
2
3
x
1
2
x
1
2
1
x
1
x
1
2
x
1
x
1
1
2
x
1
2
x
1
2
x
1
2
3
x
1
1
2
x
1
1
2


1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x
1
x


In [589]:
#X_treino = df_abt.query('GAME_DATE >= "2022-10-18" & GAME_DATE <= "2023-03-18"')
#X_treino = df_abt.query('GAME_DATE >= "2022-10-18" & GAME_DATE <= "2023-03-18"')
X_treino = df_abt.query('GAME_DATE >= "2021-10-19" & GAME_DATE <= "2023-02-18"')


X_treino.shape

(2114, 38)

In [590]:
#Definindo o Target de Treino
y_treino = X_treino.filter(['WL_HOME'])
y_treino.shape

(2114, 1)

In [591]:
#Definindo o Target de Teste
#X_teste = df_abt.query('GAME_DATE > "2023-03-18"')
X_teste = df_abt.query('GAME_DATE > "2023-03-18"')
X_teste.shape

(168, 38)

In [592]:
#Definindo o Target de Treino
y_teste = X_teste.filter(['WL_HOME'])
y_teste.shape

(168, 1)

In [593]:
X_treino = X_treino.drop('WL_HOME', axis='columns')
X_teste  = X_teste.drop('WL_HOME', axis='columns')

In [594]:
X_treino.shape

(2114, 37)

In [595]:
X_teste.shape

(168, 37)

In [596]:
def LimpaVariaveis_HOME_AWAY(df,origem):
    if origem == 'HOME':
        df = df.drop('TEAM_ID_AWAY', axis='columns',errors='ignore')
        #df = df.drop('TEAM_NAME_AWAY', axis='columns')
        df = df.drop('PTS_AWAY', axis='columns',errors='ignore')
        df = df.drop('FGM_AWAY', axis='columns',errors='ignore')
        df = df.drop('FG_PCT_AWAY', axis='columns',errors='ignore')
        df = df.drop('FG3M_AWAY', axis='columns',errors='ignore')
        df = df.drop('FG3_PCT_AWAY', axis='columns',errors='ignore')
        df = df.drop('FTM_AWAY', axis='columns',errors='ignore')
        df = df.drop('FT_PCT_AWAY', axis='columns',errors='ignore')
        df = df.drop('OREB_AWAY', axis='columns',errors='ignore')
        df = df.drop('DREB_AWAY', axis='columns',errors='ignore')
        df = df.drop('PLUS_MINUS_AWAY', axis='columns',errors='ignore')
        #df = df.drop('WL_TOTVIT_CONFRONTO_AWAY', axis='columns')
        #df = df.drop('WL_TOTVIT_TEMPORADA_AWAY', axis='columns')
        #df = df.drop('WL_TOTDER_TEMPORADA_AWAY', axis='columns')
        
    else:
        df = df.drop('TEAM_ID_HOME', axis='columns',errors='ignore')
        #df = df.drop('TEAM_ABBREVIATION_HOME', axis='columns')
        #df = df.drop('TEAM_NAME_HOME', axis='columns')
        df = df.drop('PTS_HOME', axis='columns',errors='ignore')
        df = df.drop('FGM_HOME', axis='columns',errors='ignore')
        df = df.drop('FG_PCT_HOME', axis='columns',errors='ignore')
        df = df.drop('FG3M_HOME', axis='columns',errors='ignore')
        df = df.drop('FG3_PCT_HOME', axis='columns',errors='ignore')
        df = df.drop('FTM_HOME', axis='columns',errors='ignore')
        df = df.drop('FT_PCT_HOME', axis='columns',errors='ignore')
        df = df.drop('OREB_HOME', axis='columns',errors='ignore')
        df = df.drop('DREB_HOME', axis='columns',errors='ignore')
        df = df.drop('PLUS_MINUS_HOME', axis='columns',errors='ignore')
        #df = df.drop('WL_TOTVIT_CONFRONTO_HOME', axis='columns')
        #df = df.drop('WL_TOTVIT_TEMPORADA_HOME', axis='columns')
        #df = df.drop('WL_TOTDER_TEMPORADA_HOME', axis='columns')
                 
    return df

In [597]:
X_treino.shape

(2114, 37)

## Tratar a Base de Teste para obter a média das variáveis a serem utilizadas no Modelo

In [598]:
X_teste_mean = pd.DataFrame()
    
for row in X_teste.itertuples():
    Team_id_home = row.TEAM_ID_HOME
    #Team_id_home = 1610612757    
    Team_id_away = row.TEAM_ID_AWAY
    #print(row.GAME_DATE)
    
    ultimos_X_jogos = 20
    
   #X_teste_mean.query("TEAM_ID_HOME == 1610612757")
    
    #Cria dois Dataframes, um com os últimos X jogos do time Home e outro com o AWAY
    #df_TimeHome = X_treino.query('TEAM_ID_HOME == @Team_id_home').sort_values(by=['GAME_DATE'], ascending=False, inplace=False).head(ultimos_X_jogos)
    #df_TimeAway = X_treino.query('TEAM_ID_AWAY == @Team_id_away').sort_values(by=['GAME_DATE'], ascending=False, inplace=False).head(ultimos_X_jogos)
    df_TimeHome = df_abt.query('TEAM_ID_HOME == @Team_id_home and GAME_DATE < @row.GAME_DATE').sort_values(by=['GAME_DATE'], ascending=False, inplace=False).head(ultimos_X_jogos)
    df_TimeAway = df_abt.query('TEAM_ID_AWAY == @Team_id_away and GAME_DATE < @row.GAME_DATE').sort_values(by=['GAME_DATE'], ascending=False, inplace=False).head(ultimos_X_jogos)   
    
    #Cria dois Dataframes, um com a Média do Time Home e outro com a Média do time AWAY dos últimos X jogos
    df_media_home = round(pd.DataFrame([df_TimeHome.mean(axis=0,numeric_only=True)]),2)
    df_media_away = round(pd.DataFrame([df_TimeAway.mean(axis=0,numeric_only=True)]),2)
    
    
    #Exlui a coluna TEAM_ID_HOME, pois a média bagunçou o dado e cria novamente com o valor correto
    df_media_home = df_media_home.drop('TEAM_ID_HOME', axis='columns',errors='ignore')
    df_media_away = df_media_away.drop('TEAM_ID_AWAY', axis='columns',errors='ignore')
   
    df_media_away = df_media_away.drop('WL_TOTVIT_CONFRONTO_AWAY', axis='columns',errors='ignore')
    df_media_away = df_media_away.drop('WL_TOTVIT_TEMPORADA_AWAY', axis='columns',errors='ignore')
    df_media_away = df_media_away.drop('WL_TOTDER_TEMPORADA_AWAY', axis='columns',errors='ignore')
    df_media_away = df_media_away.drop('WL_TOTVIT_CONFRONTO_HOME', axis='columns',errors='ignore')
    df_media_away = df_media_away.drop('WL_TOTVIT_TEMPORADA_HOME', axis='columns',errors='ignore')
    df_media_away = df_media_away.drop('WL_TOTDER_TEMPORADA_HOME', axis='columns',errors='ignore')
    
    df_media_away = df_media_away.drop('WL_TOTVIT_CONFRONTO_AWAY', axis='columns',errors='ignore')
    df_media_away = df_media_away.drop('WL_TOTVIT_TEMPORADA_AWAY', axis='columns',errors='ignore')
    df_media_away = df_media_away.drop('WL_TOTDER_TEMPORADA_AWAY', axis='columns',errors='ignore')
    df_media_home = df_media_away.drop('WL_TOTVIT_CONFRONTO_HOME', axis='columns',errors='ignore')
    df_media_home = df_media_away.drop('WL_TOTVIT_TEMPORADA_HOME', axis='columns',errors='ignore')
    df_media_home = df_media_away.drop('WL_TOTDER_TEMPORADA_HOME', axis='columns',errors='ignore')
   
    df_media_home['TEAM_ID_HOME'] = row.TEAM_ID_HOME
    df_media_home['SEASON_ID'] = row.SEASON_ID
    df_media_home['GAME_DATE'] = row.GAME_DATE
    df_media_home['GAME_ID'] = row.GAME_ID
    df_media_home['WL_TOTVIT_CONFRONTO_HOME'] = row.WL_TOTVIT_CONFRONTO_HOME
    df_media_home['WL_TOTVIT_TEMPORADA_HOME'] = row.WL_TOTVIT_TEMPORADA_HOME
    df_media_home['WL_TOTDER_TEMPORADA_HOME'] = row.WL_TOTDER_TEMPORADA_HOME
    df_media_home['PLUS_MINUS_HOME'] = df_media_home['PTS_HOME'] - df_media_away['PTS_AWAY'] 
        
    df_media_away['TEAM_ID_AWAY'] = row.TEAM_ID_AWAY
    df_media_away['SEASON_ID'] = row.SEASON_ID
    df_media_away['GAME_DATE'] = row.GAME_DATE
    df_media_away['GAME_ID'] = row.GAME_ID
    df_media_home['WL_TOTVIT_CONFRONTO_AWAY'] = row.WL_TOTVIT_CONFRONTO_AWAY
    df_media_home['WL_TOTVIT_TEMPORADA_AWAY'] = row.WL_TOTVIT_TEMPORADA_AWAY
    df_media_home['WL_TOTDER_TEMPORADA_AWAY'] = row.WL_TOTDER_TEMPORADA_AWAY
    df_media_away['PLUS_MINUS_AWAY'] = df_media_away['PTS_AWAY'] - df_media_home['PTS_HOME'] 
    
    df_media_home['FGM_HOME'] = df_media_home['FGM_HOME'] * 2
    df_media_home['FG3M_HOME'] = df_media_home['FG3M_HOME'] * 3
    df_media_away['FGM_AWAY'] = df_media_away['FGM_AWAY'] * 2
    df_media_away['FG3M_AWAY'] = df_media_away['FG3M_AWAY'] * 3
    
    df_media_home = LimpaVariaveis_HOME_AWAY(df_media_home,'HOME')
    df_media_away = LimpaVariaveis_HOME_AWAY(df_media_away,'AWAY')
    
    #df_merge = pd.merge(df_media_home, df_media_away, how='inner',
    #                      on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
    
        
    df_merge = pd.merge(df_media_home, df_media_away, how='inner',
                          on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
    
    X_teste_mean = pd.concat([X_teste_mean, df_merge],ignore_index=True)
     
    #break

In [599]:
X_teste_mean

,TEAM_ID_HOME,PTS_HOME,FGM_HOME,FG_PCT_HOME,FG3M_HOME,FG3_PCT_HOME,FTM_HOME,FT_PCT_HOME,OREB_HOME,DREB_HOME,PLUS_MINUS_HOME,SEASON_ID,GAME_DATE,GAME_ID,WL_TOTVIT_CONFRONTO_HOME,WL_TOTVIT_TEMPORADA_HOME,WL_TOTDER_TEMPORADA_HOME,WL_TOTVIT_CONFRONTO_AWAY,WL_TOTVIT_TEMPORADA_AWAY,WL_TOTDER_TEMPORADA_AWAY,PTS_AWAY,FGM_AWAY,FG_PCT_AWAY,FG3M_AWAY,FG3_PCT_AWAY,FTM_AWAY,FT_PCT_AWAY,OREB_AWAY,DREB_AWAY,PLUS_MINUS_AWAY,TEAM_ID_AWAY
0,1610612757,122.90,90.4,0.49,42.15,0.39,18.45,0.77,11.40,31.90,3.75,22022,2023-04-09,0022201230,1.0,34.0,47.0,1.0,49.0,32.0,119.15,85.7,0.48,49.20,0.38,17.05,0.81,10.90,32.60,-3.75,1610612744
1,1610612756,117.55,86.3,0.49,37.80,0.39,18.65,0.78,10.30,34.35,2.60,22022,2023-04-09,0022201229,2.0,60.0,21.0,1.0,44.0,37.0,114.95,82.5,0.47,37.50,0.37,19.95,0.80,10.15,32.40,-2.60,1610612746
2,1610612747,121.40,87.6,0.47,43.20,0.38,19.40,0.78,10.85,30.45,2.10,22022,2023-04-09,0022201228,2.0,43.0,38.0,1.0,47.0,34.0,119.30,89.4,0.50,35.25,0.34,18.15,0.81,10.85,35.05,-2.10,1610612762
3,1610612743,117.50,85.8,0.48,41.70,0.40,17.80,0.79,9.10,33.20,-5.15,22022,2023-04-09,0022201227,2.0,56.0,25.0,0.0,42.0,39.0,122.65,88.0,0.49,44.85,0.38,19.70,0.78,10.15,34.00,5.15,1610612758
4,1610612760,118.65,83.9,0.47,41.25,0.37,21.00,0.80,10.75,35.80,3.80,22022,2023-04-09,0022201226,1.0,35.0,46.0,1.0,39.0,42.0,114.85,84.8,0.47,38.10,0.36,17.35,0.76,10.45,33.20,-3.80,1610612763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,1610612745,116.60,84.1,0.49,36.45,0.34,20.35,0.79,9.60,34.20,7.10,22022,2023-03-19,0022201067,2.0,23.0,54.0,1.0,48.0,28.0,109.50,81.5,0.48,28.35,0.33,18.55,0.78,9.15,31.15,-7.10,1610612740
164,1610612765,111.55,79.4,0.48,39.45,0.39,19.00,0.79,8.65,33.20,3.05,22022,2023-03-19,0022201066,1.0,22.0,56.0,1.0,38.0,38.0,108.50,78.5,0.45,32.25,0.32,19.25,0.80,9.90,30.45,-3.05,1610612748
165,1610612759,120.25,88.5,0.49,38.25,0.37,19.00,0.79,11.20,33.40,0.60,22022,2023-03-19,0022201065,0.0,28.0,50.0,1.0,45.0,32.0,119.65,89.8,0.50,34.05,0.38,18.50,0.81,10.10,32.85,-0.60,1610612737
166,1610612760,111.40,79.4,0.46,35.85,0.34,20.05,0.77,9.90,31.95,0.95,22022,2023-03-19,0022201064,1.0,33.0,44.0,1.0,29.0,47.0,110.45,82.0,0.47,33.15,0.37,17.40,0.79,12.20,33.00,-0.95,1610612756


In [600]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [601]:
#Transformar o Target de W/L para 0/1
y_treino = y_treino['WL_HOME'].replace({'W': 1, 'L': 0})
y_teste = y_teste['WL_HOME'].replace({'W': 1, 'L': 0})

In [602]:
#Excluir do dataset de Treino as colunas categóricas sem relevância
X_treino = X_treino.drop('TEAM_ABBREVIATION_HOME', axis='columns')
X_treino = X_treino.drop('TEAM_ABBREVIATION_AWAY', axis='columns')
X_treino = X_treino.drop('TEAM_NAME_HOME', axis='columns')
X_treino = X_treino.drop('TEAM_NAME_AWAY', axis='columns')
X_treino = X_treino.drop('MATCHUP_HOME', axis='columns')
X_treino = X_treino.drop('MATCHUP_AWAY', axis='columns')
X_treino = X_treino.drop('GAME_DATE', axis='columns')

In [603]:
#Excluir do dataset de Teste as colunas categóricas sem relevância
#X_teste = X_teste.drop('TEAM_ABBREVIATION_HOME', axis='columns')
#X_teste = X_teste.drop('TEAM_ABBREVIATION_AWAY', axis='columns')
#X_teste = X_teste.drop('TEAM_NAME_HOME', axis='columns')
#X_teste = X_teste.drop('TEAM_NAME_AWAY', axis='columns')
#X_teste = X_teste.drop('MATCHUP_HOME', axis='columns')
#X_teste = X_teste.drop('MATCHUP_AWAY', axis='columns')
#X_teste = X_teste.drop('GAME_DATE', axis='columns')

In [604]:
#Excluir do dataset de Teste as colunas categóricas sem relevância
X_teste_mean = X_teste_mean.drop('TEAM_ABBREVIATION_HOME', axis='columns',errors='ignore')
X_teste_mean = X_teste_mean.drop('TEAM_ABBREVIATION_AWAY', axis='columns',errors='ignore')
X_teste_mean = X_teste_mean.drop('TEAM_NAME_HOME', axis='columns',errors='ignore')
X_teste_mean = X_teste_mean.drop('TEAM_NAME_AWAY', axis='columns',errors='ignore')
X_teste_mean = X_teste_mean.drop('MATCHUP_HOME', axis='columns',errors='ignore')
X_teste_mean = X_teste_mean.drop('MATCHUP_AWAY', axis='columns',errors='ignore')
X_teste_mean = X_teste_mean.drop('GAME_DATE', axis='columns',errors='ignore')

In [605]:
X_treino.shape

(2114, 30)

In [606]:
X_teste_mean.shape

(168, 30)

# Normalização dos Dados

In [607]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

min_max_scale = preprocessing.MinMaxScaler().fit(X_treino)
X_train_minmax = min_max_scale.transform(X_treino)
X_test_minmax = min_max_scale.transform(X_teste_mean)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_treino)
X_test_scaled = scaler.transform(X_teste_mean)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


In [608]:
#from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

del modelo
del modeloNorm
del modeloNormScaler
del modelo4

modelo = tree.DecisionTreeClassifier()
modeloNorm = tree.DecisionTreeClassifier()
modeloNormScaler = tree.DecisionTreeClassifier()
modelo4 = CatBoostClassifier()

modelo.fit(X_treino, y_treino)
modeloNorm.fit(X_train_minmax, y_treino)
modeloNormScaler.fit(X_train_scaled, y_treino)
modelo4.fit(X_treino, y_treino)

Learning rate set to 0.014183
0:	learn: 0.6564449	total: 3.3ms	remaining: 3.3s
1:	learn: 0.6165049	total: 6.75ms	remaining: 3.37s
2:	learn: 0.5786892	total: 10.2ms	remaining: 3.38s
3:	learn: 0.5450438	total: 13.3ms	remaining: 3.3s
4:	learn: 0.5132778	total: 16.6ms	remaining: 3.31s
5:	learn: 0.4815435	total: 19.1ms	remaining: 3.17s
6:	learn: 0.4513252	total: 20.6ms	remaining: 2.92s
7:	learn: 0.4224333	total: 22.9ms	remaining: 2.84s
8:	learn: 0.3971543	total: 25.5ms	remaining: 2.81s
9:	learn: 0.3745531	total: 28ms	remaining: 2.77s
10:	learn: 0.3528400	total: 30.7ms	remaining: 2.76s
11:	learn: 0.3322086	total: 33.2ms	remaining: 2.73s
12:	learn: 0.3136995	total: 35.6ms	remaining: 2.7s
13:	learn: 0.2946211	total: 38.1ms	remaining: 2.68s
14:	learn: 0.2764965	total: 40.9ms	remaining: 2.68s
15:	learn: 0.2609184	total: 43.6ms	remaining: 2.68s
16:	learn: 0.2449918	total: 46.1ms	remaining: 2.67s
17:	learn: 0.2305053	total: 48.6ms	remaining: 2.65s
18:	learn: 0.2183914	total: 51.2ms	remaining: 2.64

201:	learn: 0.0024970	total: 529ms	remaining: 2.09s
202:	learn: 0.0024636	total: 532ms	remaining: 2.09s
203:	learn: 0.0024434	total: 535ms	remaining: 2.08s
204:	learn: 0.0024434	total: 537ms	remaining: 2.08s
205:	learn: 0.0024396	total: 540ms	remaining: 2.08s
206:	learn: 0.0024139	total: 543ms	remaining: 2.08s
207:	learn: 0.0024111	total: 546ms	remaining: 2.08s
208:	learn: 0.0024111	total: 548ms	remaining: 2.08s
209:	learn: 0.0024085	total: 551ms	remaining: 2.07s
210:	learn: 0.0024085	total: 553ms	remaining: 2.07s
211:	learn: 0.0023804	total: 555ms	remaining: 2.06s
212:	learn: 0.0023780	total: 558ms	remaining: 2.06s
213:	learn: 0.0023579	total: 561ms	remaining: 2.06s
214:	learn: 0.0023305	total: 563ms	remaining: 2.06s
215:	learn: 0.0023277	total: 565ms	remaining: 2.05s
216:	learn: 0.0023034	total: 568ms	remaining: 2.05s
217:	learn: 0.0023024	total: 570ms	remaining: 2.04s
218:	learn: 0.0023004	total: 573ms	remaining: 2.04s
219:	learn: 0.0022982	total: 575ms	remaining: 2.04s
220:	learn: 

411:	learn: 0.0012583	total: 1.06s	remaining: 1.51s
412:	learn: 0.0012583	total: 1.06s	remaining: 1.51s
413:	learn: 0.0012583	total: 1.06s	remaining: 1.5s
414:	learn: 0.0012583	total: 1.06s	remaining: 1.5s
415:	learn: 0.0012581	total: 1.07s	remaining: 1.5s
416:	learn: 0.0012581	total: 1.07s	remaining: 1.5s
417:	learn: 0.0012581	total: 1.07s	remaining: 1.49s
418:	learn: 0.0012575	total: 1.07s	remaining: 1.49s
419:	learn: 0.0012575	total: 1.08s	remaining: 1.49s
420:	learn: 0.0012392	total: 1.08s	remaining: 1.49s
421:	learn: 0.0012311	total: 1.08s	remaining: 1.48s
422:	learn: 0.0012311	total: 1.08s	remaining: 1.48s
423:	learn: 0.0012311	total: 1.09s	remaining: 1.48s
424:	learn: 0.0012189	total: 1.09s	remaining: 1.47s
425:	learn: 0.0012189	total: 1.09s	remaining: 1.47s
426:	learn: 0.0012189	total: 1.09s	remaining: 1.47s
427:	learn: 0.0012189	total: 1.1s	remaining: 1.47s
428:	learn: 0.0012189	total: 1.1s	remaining: 1.46s
429:	learn: 0.0012189	total: 1.1s	remaining: 1.46s
430:	learn: 0.00120

620:	learn: 0.0010865	total: 1.58s	remaining: 967ms
621:	learn: 0.0010865	total: 1.59s	remaining: 965ms
622:	learn: 0.0010865	total: 1.59s	remaining: 962ms
623:	learn: 0.0010865	total: 1.59s	remaining: 960ms
624:	learn: 0.0010865	total: 1.59s	remaining: 957ms
625:	learn: 0.0010865	total: 1.6s	remaining: 954ms
626:	learn: 0.0010865	total: 1.6s	remaining: 952ms
627:	learn: 0.0010865	total: 1.6s	remaining: 949ms
628:	learn: 0.0010865	total: 1.6s	remaining: 947ms
629:	learn: 0.0010864	total: 1.61s	remaining: 944ms
630:	learn: 0.0010864	total: 1.61s	remaining: 941ms
631:	learn: 0.0010864	total: 1.61s	remaining: 939ms
632:	learn: 0.0010864	total: 1.61s	remaining: 936ms
633:	learn: 0.0010864	total: 1.62s	remaining: 933ms
634:	learn: 0.0010864	total: 1.62s	remaining: 930ms
635:	learn: 0.0010863	total: 1.62s	remaining: 928ms
636:	learn: 0.0010863	total: 1.62s	remaining: 925ms
637:	learn: 0.0010863	total: 1.63s	remaining: 922ms
638:	learn: 0.0010863	total: 1.63s	remaining: 920ms
639:	learn: 0.00

824:	learn: 0.0010793	total: 2.11s	remaining: 449ms
825:	learn: 0.0010793	total: 2.12s	remaining: 446ms
826:	learn: 0.0010793	total: 2.12s	remaining: 443ms
827:	learn: 0.0010793	total: 2.12s	remaining: 441ms
828:	learn: 0.0010793	total: 2.12s	remaining: 438ms
829:	learn: 0.0010793	total: 2.13s	remaining: 436ms
830:	learn: 0.0010789	total: 2.13s	remaining: 433ms
831:	learn: 0.0010789	total: 2.13s	remaining: 431ms
832:	learn: 0.0010789	total: 2.13s	remaining: 428ms
833:	learn: 0.0010789	total: 2.14s	remaining: 425ms
834:	learn: 0.0010789	total: 2.14s	remaining: 423ms
835:	learn: 0.0010789	total: 2.14s	remaining: 420ms
836:	learn: 0.0010789	total: 2.14s	remaining: 417ms
837:	learn: 0.0010789	total: 2.15s	remaining: 415ms
838:	learn: 0.0010789	total: 2.15s	remaining: 412ms
839:	learn: 0.0010789	total: 2.15s	remaining: 410ms
840:	learn: 0.0010789	total: 2.15s	remaining: 407ms
841:	learn: 0.0010785	total: 2.15s	remaining: 404ms
842:	learn: 0.0010785	total: 2.16s	remaining: 402ms
843:	learn: 

In [609]:
X_treino

,SEASON_ID,TEAM_ID_HOME,GAME_ID,PTS_HOME,FGM_HOME,FG_PCT_HOME,FG3M_HOME,FG3_PCT_HOME,FTM_HOME,FT_PCT_HOME,OREB_HOME,DREB_HOME,PLUS_MINUS_HOME,TEAM_ID_AWAY,PTS_AWAY,FGM_AWAY,FG_PCT_AWAY,FG3M_AWAY,FG3_PCT_AWAY,FTM_AWAY,FT_PCT_AWAY,OREB_AWAY,DREB_AWAY,PLUS_MINUS_AWAY,WL_TOTVIT_CONFRONTO_HOME,WL_TOTVIT_CONFRONTO_AWAY,WL_TOTVIT_TEMPORADA_HOME,WL_TOTDER_TEMPORADA_HOME,WL_TOTVIT_TEMPORADA_AWAY,WL_TOTDER_TEMPORADA_AWAY
346,22022,1610612756,0022200885,107,42,0.472,11,0.355,12,0.750,7,31,-9.0,1610612746,116,40,0.488,11,0.324,25,0.926,8,35,9.0,2.0,0.0,52.0,18.0,39.0,34.0
347,22022,1610612750,0022200884,106,39,0.411,8,0.276,20,0.909,16,37,-8.0,1610612764,114,43,0.453,14,0.400,14,0.824,13,36,8.0,0.0,1.0,46.0,27.0,45.0,27.0
348,22022,1610612741,0022200883,100,37,0.381,10,0.313,16,0.762,15,36,-12.0,1610612749,112,40,0.455,18,0.383,14,0.778,9,43,12.0,1.0,2.0,43.0,26.0,29.0,40.0
349,22022,1610612747,0022200882,120,44,0.530,10,0.270,22,0.611,6,37,18.0,1610612740,102,39,0.438,7,0.280,17,0.708,13,37,-18.0,1.0,2.0,34.0,34.0,42.0,27.0
350,22022,1610612743,0022200881,118,48,0.571,11,0.355,11,0.733,6,37,9.0,1610612742,109,39,0.464,14,0.359,17,0.850,6,25,-9.0,1.0,1.0,49.0,22.0,36.0,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,22021,1610612752,0022100005,138,51,0.486,17,0.378,19,0.704,7,48,4.0,1610612738,134,48,0.410,21,0.368,17,0.739,15,41,-4.0,1.0,0.0,1.0,0.0,0.0,1.0
2456,22021,1610612765,0022100004,88,36,0.400,6,0.214,10,0.769,11,36,-6.0,1610612741,94,37,0.430,7,0.304,13,0.867,9,39,6.0,0.0,1.0,0.0,1.0,1.0,0.0
2457,22021,1610612766,0022100003,123,46,0.430,13,0.419,18,0.667,12,34,1.0,1610612754,122,42,0.467,17,0.362,21,0.875,8,43,-1.0,1.0,0.0,1.0,0.0,0.0,1.0
2458,22021,1610612747,0022100002,114,45,0.474,15,0.357,9,0.474,5,40,-7.0,1610612744,121,41,0.441,14,0.359,25,0.833,9,41,7.0,0.0,1.0,0.0,1.0,1.0,0.0


In [610]:
y_pred = modelo.predict(X_teste_mean)
y_predNorm = modeloNorm.predict(X_test_minmax)
y_predNormScaler = modeloNormScaler.predict(X_test_scaled)
y_pred4 = modelo4.predict(X_teste_mean)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


In [611]:
from sklearn.metrics import accuracy_score

In [612]:
accuracy_score(y_teste, y_pred)
#0.4430379746835443

0.5535714285714286

In [613]:
accuracy_score(y_teste, y_predNorm)

0.5535714285714286

In [614]:
accuracy_score(y_teste, y_predNormScaler)

0.5535714285714286

In [615]:
accuracy_score(y_teste, y_pred4)

0.5714285714285714

In [ ]:
X_teste_mean.query("TEAM_ID_HOME == 1610612757")

In [ ]:
BaseTreino = X_teste_mean

In [ ]:
BaseTreino['WL_HOME'] = y_teste

In [ ]:
BaseTreino['WL_HOME_Previsto'] = y_pred4

In [ ]:
BaseTreino.head(5)
#df_abt.query("TEAM_ID_HOME == 1610612757")

In [ ]:
X_teste.head(5)

In [ ]:
df_abt.query("GAME_ID == '0022201230'")
#WL_HOME=L
#PTS_HOME=101
#PTS_AWAY=157
#PLUS_MINUS_HOME=-56.0
#PLUS_MINUS_AWAY=56.0

In [ ]:
tempH = df_abt.query("TEAM_ID_HOME == 1610612757")
tempA = df_abt.query("TEAM_ID_AWAY == 1610612744")

In [ ]:
round(pd.DataFrame([tempH.mean(axis=0,numeric_only=True)]),2)

In [ ]:
round(pd.DataFrame([tempA.mean(axis=0,numeric_only=True)]),2)

In [ ]:
    df_TimeHome = X_treino.query('TEAM_ID_HOME == @Team_id_home').sort_values(by=['GAME_DATE'], ascending=False, inplace=False).head(ultimos_X_jogos)
    df_TimeAway = X_treino.query('TEAM_ID_AWAY == @Team_id_away').sort_values(by=['GAME_DATE'], ascending=False, inplace=False).head(ultimos_X_jogos)
   
    #Cria dois Dataframes, um com a Média do Time Home e outro com a Média do time AWAY dos últimos X jogos
    df_media_home = round(pd.DataFrame([df_TimeHome.mean(axis=0,numeric_only=True)]),2)
    df_media_away = round(pd.DataFrame([df_TimeAway.mean(axis=0,numeric_only=True)]),2)

In [ ]:
from sklearn.metrics import confusion_matrix
mat_conf = confusion_matrix(y_teste, y_pred)
print("Matriz de confusão:")
print(mat_conf)

In [ ]:
df_abt

In [ ]:
df_cm = pd.DataFrame(mat_conf, columns=['false', 'positive'], index=['false', 'positive'])
df_cm

In [ ]:
df_cm

In [ ]:
import seaborn as sns
fig, ax = plt.subplots(figsize=(10,6))
sns.heatmap(df_cm, cmap='Blues', annot=True, annot_kws={'size': 20}, ax = ax, fmt='d');

In [ ]:
sns.heatmap(mat_conf, annot=True,fmt='g')

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

# Change figure size and increase dpi for better resolution
# and get reference to axes object
fig, ax = plt.subplots(figsize=(8,6), dpi=100)

# initialize using the raw 2D confusion matrix 
# and output labels (in our case, it's 0 and 1)
display = ConfusionMatrixDisplay(mat_conf, display_labels=modelo.classes_)

# set the plot title using the axes object
ax.set(title='Confusion Matrix')

# show the plot. 
# Pass the parameter ax to show customizations (ex. title) 
display.plot(ax=ax);

In [ ]:
import seaborn as sns

# Change figure size and increase dpi for better resolution
plt.figure(figsize=(8,6), dpi=100)
# Scale up the size of all text
sns.set(font_scale = 1.1)

# Plot Confusion Matrix using Seaborn heatmap()
# Parameters:
# first param - confusion matrix in array format   
# annot = True: show the numbers in each heatmap cell
# fmt = 'd': show numbers as integers. 
ax = sns.heatmap(mat_conf, annot=True, fmt='d', )

# set x-axis label and ticks. 
ax.set_xlabel("Predicted Diagnosis", fontsize=14, labelpad=20)
ax.xaxis.set_ticklabels(['0', '1'])

# set y-axis label and ticks
ax.set_ylabel("Actual Diagnosis", fontsize=14, labelpad=20)
ax.yaxis.set_ticklabels(['0', '1'])

# set plot title
ax.set_title("Confusion Matrix", fontsize=14, pad=20)

plt.show()

In [ ]:
y_pred
#numpy.ndarray

In [ ]:
from sklearn.metrics import classification_report
relatorio = classification_report(y_teste, y_pred)
print("Relatório de classificação:")
print(relatorio)

In [ ]:
comparacao = pd.DataFrame(data={'real': y_teste, 
                                'prev': modelo.predict(X_teste).squeeze()})

In [ ]:
comparacao

In [ ]:
comparacao['dif'] = abs(comparacao.real - comparacao.prev)
comparacao

In [ ]:
comparacao.dif.mean()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
normalizador = MinMaxScaler()
normalizador.fit(X_treino)
X_treino_norm = normalizador.transform(X_treino)
X_teste_norm = normalizador.transform(X_teste)

In [ ]:
X_treino_norm

## Análise Exploratória


In [ ]:
#Tamanho do Dataframe (ABT)
df_abt.shape

In [ ]:
#Registros Únicos por campo
df_abt.nunique()

In [ ]:
#Analisando se há colunas com Null
df_abt.isnull().sum()

In [ ]:
df_abt.isnull().sum().plot()

In [ ]:
# Vitórias/Derrota do time que jogou em Casa (qtd)
df_abt['WL_HOME'].value_counts()

In [ ]:
# Vitórias/Derrota do time que jogou em Casa (%)
df_abt['WL_HOME'].value_counts(normalize=True)


In [ ]:
TabelaPercentual = df_abt['WL_HOME'].value_counts(normalize=True)
TabelaPercentual.plot.pie(autopct='%1.1f%%',figsize=(11, 5))

In [ ]:
#Início e Fim da Temporada regular
df_abt.agg({'GAME_DATE': ['min', 'max']}) 

In [ ]:
df_abt['TEAM_NAME_HOME'].value_counts().sort_index(ascending=True) #Jogos por time em casa

In [ ]:
df_total_jogos = pd.DataFrame(df_abt['TEAM_NAME_HOME'].value_counts().sort_index(ascending=True)).reset_index() #Jogos por time

In [ ]:
df_total_jogos

In [ ]:
df_total_jogos.rename(columns = {'index': 'Time', 'TEAM_NAME_HOME': 'Qtd Jogos'}, inplace = True)

In [ ]:
df_total_jogos

In [ ]:
df_abt['TEAM_NAME_HOME'].value_counts().sort_index(ascending=True).plot(kind="barh",rot=50, fontsize=6, y="Qtd. Jogos", x="Time")

In [ ]:
import matplotlib.pyplot as plt

time = df_total_jogos['Time']
total_jogos = df_total_jogos['Qtd Jogos']

In [ ]:
fig = plt.figure(figsize = (7, 11))
 
# creating the bar plot
plt.barh(time,total_jogos, height = 0.75,color='c')
plt.xlabel("Total de Jogos")
plt.ylabel("Time")
plt.title("Total de Jogos x Time na temporada Regular 22/23")
plt.show

In [ ]:
df_abt[['TEAM_NAME_HOME','WL_HOME']]

In [ ]:
agrupado = df_abt.groupby(['TEAM_NAME_HOME','WL_HOME'])['TEAM_NAME_HOME'].count().rename('Total').reset_index()

In [ ]:
agrupado

In [ ]:
df_perdedores = agrupado[(agrupado['WL_HOME'] == 'L')].sort_values(by=['Total'], ascending=False)

In [ ]:
df_perdedores.head(5)
df_perdedores = df_perdedores.drop('WL_HOME', axis='columns')


In [ ]:
df_perdedores

In [ ]:
fig = plt.figure(figsize = (9, 5))

time = df_perdedores['TEAM_NAME_HOME'].head(5)
total = df_perdedores['Total'].head(5)
 
# creating the bar plot
plt.bar(time, total, color='c')
plt.xlabel("Total de Jogos")
plt.ylabel("Time")
plt.title("Total de Jogos x Time na temporada Regular 22/23")
plt.xticks(rotation ='50')
plt.xticks(x, labels, rotation ='vertical')
plt.show

In [ ]:
time = df_perdedores['TEAM_NAME_HOME'].head(5)
total = df_perdedores['Total'].head(5)

# Create Tick Locations
x = np.arange(len(time))

# Plotting
fig, ax = plt.subplots(figsize=(9, 5))

# Do not offset the values of x to center labels (default)
ax.bar(x, total, width=.50, label='total',color = 'r')

# Labels, ticks, etc.
ax.set_ylabel('Qtd.')
ax.set_title('Total de Derrotas jogando em Casa (TOP 5)')
ax.set_xticks(x)
ax.set_xticklabels(time)

# Add Bar Labels
for c in ax.containers:
    ax.bar_label(c)

plt.xticks(rotation ='40')
plt.show()

In [ ]:
time = df_perdedores['TEAM_NAME_HOME'].head(5)
total = df_perdedores['Total'].head(5)

# Create Tick Locations
x = np.arange(len(time))

# Plotting
fig, ax = plt.subplots(figsize=(9, 5))

# Do not offset the values of x to center labels (default)
ax.bar(x, round(total/41,2), width=.50, label='total',color = 'r')

# Labels, ticks, etc.
ax.set_ylabel('Qtd.')
ax.set_title('% de Derrota jogando em Casa (TOP 5)')
ax.set_xticks(x)
ax.set_xticklabels(time)

# Add Bar Labels
for c in ax.containers:
    ax.bar_label(c)

plt.xticks(rotation ='40')
plt.show()

In [ ]:
df_perdedores.head(5)

In [ ]:
df_ganhadores = agrupado[(agrupado['WL_HOME'] == 'W')].sort_values(by=['Total'], ascending=False)
df_ganhadores = df_ganhadores.drop('WL_HOME', axis='columns')
df_ganhadores.head(5)

In [ ]:
time = df_ganhadores['TEAM_NAME_HOME'].head(5)
total = df_ganhadores['Total'].head(5)

# Create Tick Locations
x = np.arange(len(time))

# Plotting
fig, ax = plt.subplots(figsize=(9, 5))

# Do not offset the values of x to center labels (default)
ax.bar(x, total, width=.50, label='total',color = 'g')

# Labels, ticks, etc.
ax.set_ylabel('Qtd.')
ax.set_title('Total de Vitórias jogando em Casa (TOP 5)')
ax.set_xticks(x)
ax.set_xticklabels(time)

# Add Bar Labels
for c in ax.containers:
    ax.bar_label(c)

plt.xticks(rotation ='40')
plt.show()

In [ ]:
time = df_ganhadores['TEAM_NAME_HOME'].head(5)
total = df_ganhadores['Total'].head(5)

# Create Tick Locations
x = np.arange(len(time))

# Plotting
fig, ax = plt.subplots(figsize=(9, 5))

# Do not offset the values of x to center labels (default)
ax.bar(x, round(total/41,2), width=.50, label='total',color = 'g')

# Labels, ticks, etc.
ax.set_ylabel('Qtd.')
ax.set_title('% de Vitórias jogando em Casa (TOP 5)')
ax.set_xticks(x)
ax.set_xticklabels(time)

# Add Bar Labels
for c in ax.containers:
    ax.bar_label(c)

plt.xticks(rotation ='40')
plt.show()

In [ ]:
df_abt

# Análise dados quantitativos

In [ ]:
import seaborn as srn

In [ ]:
df_abt.describe()

In [ ]:
# Var PTS_HOME
df_abt.PTS_HOME.describe()

In [ ]:
# Var PTS_AWAY
df_abt.PTS_AWAY.describe()

In [ ]:
df_abt[['PTS_HOME','PTS_AWAY']]

In [ ]:
df_pontos = pd.concat([df_abt.PTS_HOME.describe(), df_abt.PTS_AWAY.describe()], axis=1)
df_pontos

In [ ]:
srn.boxplot(df_abt['PTS_HOME'],orient="v")


In [ ]:
plt.figure(figsize =(10, 6))
df = pd.concat([df_abt['PTS_HOME'], df_abt['PTS_AWAY']], axis=1)
srn.boxplot(data=df)

In [ ]:
df

In [ ]:
df_pontos = df_abt.filter(['PTS_HOME','PTS_AWAY'])
df_pontos

In [ ]:

# Tamanho do gráfico em polegadas
plt.figure(figsize =(10, 6))

# Cores dos boxplot
colors = ['Oranges', 'g', 'r']

# plotviolin das espécies em relação ao tamanho das sépalas
ax = sns.boxplot( x = "PTS_HOME",data = df_abt, linewidth=5, palette='Oranges')

# Gráfico de disperção
ax = sns.stripplot(x = "PTS_HOME", hue="PTS_HOME" ,data = df_abt,palette='Dark2')  

# Adicionando Título ao gráfico
plt.title("Boxplot Total de Pontos do time da Casa", loc="center", fontsize=18)
plt.xlabel("Pontos")
plt.ylabel("")

plt.show()

In [ ]:
#Pontos convertidos de 2, 3 e lance livre
#Time da Casa

df_pontos = pd.concat(
    [df_abt.FGM_HOME.describe(),
     df_abt.FG3M_HOME.describe(),
     df_abt.FTM_HOME.describe()
    ],
     axis=1)
df_pontos

In [ ]:
plt.figure(figsize =(10, 6))
df = pd.concat([df_abt['FGM_HOME'], df_abt['FG3M_HOME'],df_abt['FTM_HOME']], axis=1)
srn.boxplot(data=df)

In [ ]:
#Pontos convertidos de 2, 3 e lance livre
#Time fora de Casa

df_pontos = pd.concat(
    [df_abt.FGM_AWAY.describe(),
     df_abt.FG3M_AWAY.describe(),
     df_abt.FTM_AWAY.describe()
    ],
     axis=1)
df_pontos

In [ ]:
plt.figure(figsize =(10, 6))
df = pd.concat([df_abt['FGM_AWAY'], df_abt['FG3M_AWAY'],df_abt['FTM_AWAY']], axis=1)
srn.boxplot(data=df)

In [ ]:
#Rebotes Ofensivos e Defensivos
#Time da Casa e fora de Casa

df_pontos = pd.concat(
    [df_abt.OREB_HOME.describe(),
     df_abt.OREB_AWAY.describe(),
     df_abt.DREB_HOME.describe(),
     df_abt.DREB_AWAY.describe()
    ],
     axis=1)
df_pontos

In [ ]:
plt.figure(figsize =(10, 6))
df = pd.concat([df_abt['OREB_HOME'], df_abt['OREB_AWAY'],df_abt['DREB_HOME'],df_abt['DREB_AWAY']], axis=1)
srn.boxplot(data=df)

# Análise de Correlação

In [ ]:
correlation = df_abt.corr()
correlation

In [ ]:

# plot da matriz de correlação
plot = sns.heatmap(correlation, annot = True, fmt=".1f", linewidths=.6)
plot

# Deletar da Base

In [ ]:
WL_AWAY

In [ ]:
name = df['car'].head(12)
price = df['price'].head(12)
 
# Figure Size
fig = plt.figure(figsize =(10, 7))
 
# Horizontal Bar Plot
plt.bar(name[0:10], price[0:10])
 
# Show Plot
plt.show()